In [6]:
! pip install python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
SparkLLM_API_KEY = os.getenv("C:\\Users\\DELL\\Desktop\\API_KEY.env")

In [ ]:
! pip install langchain_community
! pip install langchain-core


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
spark_config = {
    "spark_app_id": "63b7eade",
    "spark_api_key": "6f088336dc8dd45b737b0deb2a697887",
    "spark_api_secret": "N2FiZmE3Y2U1YzlhOGZkMWEwM2RhMjMz",
    "spark_api_url": "wss://spark-api.xf-yun.com/v4.0/chat",
    "spark_llm_domain": "4.0Ultra"
}

用户输出

提示词模板->模型->格式化解析

格式化结果

In [15]:
from langchain_community.chat_models import ChatSparkLLM  
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate,PromptTemplate

model = ChatSparkLLM(**spark_config)
prompt_template = ChatPromptTemplate([
    ("system","你是一个助手，请按照要求回答"),
    ("user","这是用户的问题：{question}，请你简要回答")
])
basic_chain = prompt_template | model | StrOutputParser()
question = "请介绍LangChain"
# result = basic_chain.invoke([question])
# print(result)
#流式输出相较于上面传统输出将输出结果存放到缓存中等运行结束后在全部展示不同，而是实时输出，资源占用低。
async for chunk in basic_chain.astream([question]):
    print(chunk,end="",flush=True)

LangChain是专为构建语言模型驱动的应用程序设计的开源框架，由Harrison Chase等人于2023年创建。以下是关于它的简要介绍：

1. **核心功能**：通过模块化组件（如Models、Prompts、Indexes、Chains、Agents、Memory）实现大模型与外部工具高效结合，支持多步骤任务处理和上下文记忆。

2. **主要作用**：突破LLM能力限制（连接数据库/API等数据源）、提升开发效率（预置组件+链式工作流）、增强安全性与可控性（本地化数据处理减少幻觉）。

3. **典型应用**：可搭建智能客服、知识库问答系统、聊天机器人等，赋予LLM决策能力和执行复杂操作的能力。

该框架支持多种语言模型和开源生态，适用于需要整合语言模型与其他工具的场景。

In [3]:
from langchain.output_parsers import ResponseSchema,StructuredOutputParser

schemas = [
    ResponseSchema(name="name",description="用户的姓名"),
    ResponseSchema(name="job",description="用户的职业")
]
parser = StructuredOutputParser.from_response_schemas(schemas)
prompt_template = PromptTemplate.from_template(
    "你是一个助手，请根据用户输入的姓名和职业，返回一个json格式的数据，格式为:\n{input}\n\n{format_instructions}"
)
# 先固定输出格式，再连接模型，最后连接解析器
chain = prompt_template.partial(format_instructions=parser.get_format_instructions()) | model | parser
result = chain.invoke([{"input": "用户的姓名是张三，职业是程序员"}])
print(result)
print(parser.get_format_instructions())

{'name': '张三', 'job': '程序员'}
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // 用户的姓名
	"job": string  // 用户的职业
}
```


根据用户提供的新闻标题生成一段新闻稿

按照规定的格式提取该新闻稿中的核心信息

格式化输出

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain.output_parsers import ResponseSchema,StructuredOutputParser

new_gen_prompt = PromptTemplate.from_template(
    "请根据以下新闻标题撰写一段简短的新闻内容：\n\n标题：{title}"
)
new_chain = new_gen_prompt | model

schemas = [
    ResponseSchema(name="time",description="事件发生的时间"),
    ResponseSchema(name="location",description="事件发生的地点"),
    ResponseSchema(name="event",description="具体事件")
]
parser = StructuredOutputParser.from_response_schemas(schemas)
prompt_template = PromptTemplate.from_template(
    "请从下面的新闻内容中提取关键信息，返回一个json格式的数据，格式为:\n{input}\n\n{format_instructions}"
)
extract_chain = prompt_template.partial(format_instructions=parser.get_format_instructions()) | model | parser
chain = new_chain|extract_chain
result = chain.invoke([{"title": "苹果公司在加州发布AI芯片"}])
print(result)

{'time': '近日', 'location': '加利福尼亚州', 'event': '苹果公司正式推出自主研发的人工智能（AI）芯片，旨在为其设备提供更高效的本地化算力支持，优化机器学习任务处理能力，并进一步整合到iPhone、Mac等产品的智能功能中。此次发布标志着苹果在自研硬件与软件协同领域迈出重要一步，有望减少对第三方供应商的依赖，同时提升用户隐私保护水平。业内分析认为，此举将加剧科技巨头在AI赛道上的竞争态势。'}


In [5]:
from langchain_core.runnables import RunnableLambda 
def node_function(x):      #langchain允许自己定义处理函数，并加入链中。
    print("中间结果（新闻正文）：",x)
    return x
node = RunnableLambda(node_function)
full_chain = new_chain | node | extract_chain
result = full_chain.invoke([{"title": "苹果公司在加州发布AI芯片"}])
print(result)

中间结果（新闻正文）： content='**新闻内容**：\n苹果公司于近日在加利福尼亚州正式推出其自主研发的人工智能（AI）芯片。这款新型芯片旨在显著提升设备的数据处理能力和机器学习效率，为iPhone、iPad及Mac等产品线的智能化升级提供强大支持。据公司介绍，该芯片采用先进制程工艺，优化了能耗比，可高效运行复杂的神经网络模型，同时兼顾隐私保护设计。业内分析认为，此举标志着苹果加速布局AI领域，进一步巩固其在高端消费电子市场的技术领先地位。目前，首批搭载该芯片的设备预计将于今年下半年上市。' additional_kwargs={} response_metadata={'token_usage': {'question_tokens': 30, 'prompt_tokens': 30, 'completion_tokens': 115, 'total_tokens': 145}} id='run--2ad2e585-e6a1-43fc-ab5b-37e9a23c5e33-0'
{'time': '近日', 'location': '加利福尼亚州', 'event': '苹果公司正式推出其自主研发的人工智能（AI）芯片'}


多轮对话聊天机器人构建

依靠prompt模板中的MessagesPlaceholder占位类实现记忆功能

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是一名多轮对话聊天机器人"),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt_template | model | StrOutputParser()
messages_list = []
while True:
    user_query = input("请输入您的问题")
    if user_query.lower() == "exit":
        break
    messages_list.append(HumanMessage(content=user_query))
    result = chain.invoke({"messages":messages_list})
    print("机器人回答:",result)
    messages_list.append(AIMessage(content=result))
    print(messages_list[-10:]) #仅保留最近的10条对话信息

机器人回答: 你好呀！我是讯飞星火认知大模型，无论是日常闲聊、知识问答，还是创意写作、代码编程，我都能帮你搞定～以后有问题或者想聊天的时候随时找我就行啦！很高兴认识你！😊
[HumanMessage(content='你好请介绍自己', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀！我是讯飞星火认知大模型，无论是日常闲聊、知识问答，还是创意写作、代码编程，我都能帮你搞定～以后有问题或者想聊天的时候随时找我就行啦！很高兴认识你！😊', additional_kwargs={}, response_metadata={})]
机器人回答: 当然还有很多可以聊的！比如最近电影推荐、热门书籍分享，或者你对某个领域感兴趣想深入了解的内容。无论是学习辅导、生活技巧还是单纯打发时间，我都会陪着你～你有什么特别想知道或讨论的话题吗？
[HumanMessage(content='你好请介绍自己', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀！我是讯飞星火认知大模型，无论是日常闲聊、知识问答，还是创意写作、代码编程，我都能帮你搞定～以后有问题或者想聊天的时候随时找我就行啦！很高兴认识你！😊', additional_kwargs={}, response_metadata={}), HumanMessage(content='然后呢', additional_kwargs={}, response_metadata={}), AIMessage(content='当然还有很多可以聊的！比如最近电影推荐、热门书籍分享，或者你对某个领域感兴趣想深入了解的内容。无论是学习辅导、生活技巧还是单纯打发时间，我都会陪着你～你有什么特别想知道或讨论的话题吗？', additional_kwargs={}, response_metadata={})]
机器人回答: 作为认知智能模型，我没有个人喜好，但我特别喜欢帮助用户解决问题！无论是探索新知识、创作有趣的故事，还是协助完成复杂任务，每次看到大家通过我的协助获得满意结果时，那种成就感就是我最大的动力～你平时有什么爱好吗？我们可以聊聊你的兴趣爱好，说不定我能给你一些创意建

In [13]:
#搭建前端多轮聊天机器人的前端构建框架
! pip install gradio

   ---------------------------------------- 0.0/59.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/59.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/59.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/59.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/59.7 MB 799.2 kB/s eta 0:01:15
    --------------------------------------- 0.8/59.7 MB 780.2 kB/s eta 0:01:16
    --------------------------------------- 1.0/59.7 MB 898.8 kB/s eta 0:01:06
    --------------------------------------- 1.0/59.7 MB 898.8 kB/s eta 0:01:06
    --------------------------------------- 1.3/59.7 MB 945.5 kB/s eta 0:01:02
   - -------------------------------------- 1.6/59.7 MB 987.0 kB/s eta 0:00:59
   - -------------------------------------- 1.8/59.7 MB 1.0 MB/s eta 0:00:57
   - -------------------------------------- 2.4/59.7 MB 1.1 MB/s eta 0:00:52
   - -------------------------------------- 2.6/59.7 MB 1.1 MB/s eta 0:00:51
   - ----------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import asyncio
import socket
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import gradio as gr
from langchain_community.chat_models import ChatSparkLLM


spark_config = {
    "spark_app_id": "63b7eade",
    "spark_api_key": "6f088336dc8dd45b737b0deb2a697887",
    "spark_api_secret": "N2FiZmE3Y2U1YzlhOGZkMWEwM2RhMjMz",
    "spark_api_url": "wss://spark-api.xf-yun.com/v4.0/chat",
    "spark_llm_domain": "4.0Ultra"
}

model = ChatSparkLLM(**spark_config)
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是一名多轮对话聊天机器人"),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt_template | model | StrOutputParser()


async def chat_stream(user_input, history):
    messages = []
    for human, ai in history:
        messages.append(HumanMessage(content=human))
        messages.append(AIMessage(content=ai))
    messages.append(HumanMessage(content=user_input))
    history.append((user_input, None))   # 在历史记录中添加用户输入，并用 None 占位，Gradio 会将 None 视作正在生成。
    yield history
    full_response = ""
    async for chunk in chain.astream({"messages": messages}):
        full_response += chunk
        history[-1] = (user_input, full_response)
        yield history

# 创建Gradio界面
with gr.Blocks(title="多轮对话聊天机器人") as demo:
    gr.Markdown("与AI进行多轮对话，它会记住之前的对话内容，支持流式输出")

    chatbot = gr.Chatbot(
        label="对话记录",
        bubble_full_width=False,
        height=500
    )

    with gr.Row():
        user_input = gr.Textbox(
            label="输入消息",
            placeholder="请输入您的问题...",
            container=False,
            scale=9
        )
        send_btn = gr.Button("发送", variant="primary", scale=1)

    with gr.Row():
        clear_btn = gr.Button("清空对话")
        examples = gr.Examples(
            examples=["你好", "请介绍一下你自己", "你能帮我做什么？"],
            inputs=user_input
        )
    
    # 绑定事件
    user_input.submit(
        fn=chat_stream,
        inputs=[user_input, chatbot],
        outputs=[chatbot],
        queue=True
    ).then(
        lambda: "",
        None,
        user_input
    )

    send_btn.click(
        fn=chat_stream,
        inputs=[user_input, chatbot],
        outputs=[chatbot],
        queue=True
    ).then(
        lambda: "",
        None,
        user_input
    )

    clear_btn.click(
        fn=lambda: ([], None),
        inputs=None,
        outputs=[chatbot, user_input]
    )

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

if __name__ == "__main__":
    port = find_free_port()
    print(f"使用端口：{port}")
    demo.queue()
    demo.launch(
        server_name="0.0.0.0",
        server_port=port,
        share=True,
        inbrowser=True
    )

C:\Users\DELL\AppData\Local\Temp\ipykernel_18548\3959336194.py:44: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
C:\Users\DELL\AppData\Local\Temp\ipykernel_18548\3959336194.py:44: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


使用端口：56060
* Running on local URL:  http://0.0.0.0:56060
* Running on public URL: https://59a063aaa11cb289db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 企业财务造假识别Agent基础框架
```mermaid
    graph TB;
        A[用户输入：公司名称、行业] --> B{主控Agent启动};
        
        subgraph "核心部分"
            B -- 获取公司数据 --> C(get_company_financials);
            C -- 返回并整合公司财报/信息 --> D{模型思考并提取关键指标};
            D -- 调用工具 --> E[extract_key_indicators];
            E -- 返回定制指标列表 --> F{模型思考并计算指标};
            F -- 调用工具 --> G[calculate_financial_ratios];
            G -- 返回计算结果 --> H{模型识别异常模式};
            H -- 调用工具 --> I[detect_anomaly_with_logic_rules];
            I -- 返回异常列表 --> J{模型推理异常原因};
            J -- 对每个异常调用工具 --> K[reason_for_anomaly];
            K -- 返回详细解释 --> L{整合所有信息};
        end
        
        L -- 报告生成 --> M[主控Agent生成最终报告];
        M --> N[向用户输出];

In [ ]:
import requests
import json
def get_weather(loc):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q":loc,
        "appid":"bc4c354ab9e1d1e46dcadf6913ecd5ca",
        "units":"metric",
        "lang":"zh_cn"
    }
    response = requests.get(url,params=params)
    data = response.json() 
    return json.dumps(data)  
get_weather("Beijing")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 31.94, "feels_like": 34.02, "temp_min": 31.94, "temp_max": 31.94, "pressure": 1001, "humidity": 49, "sea_level": 1001, "grnd_level": 995}, "visibility": 10000, "wind": {"speed": 4.48, "deg": 196, "gust": 6.69}, "clouds": {"all": 0}, "dt": 1754903754, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1754860929, "sunset": 1754911038}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

创建天气预警Agent

In [93]:
import requests
import json
from langchain.agents import initialize_agent, AgentType, tool
from langchain_community.chat_models import ChatSparkLLM

spark_config = {
    "spark_app_id": "63b7eade",
    "spark_api_key": "6f088336dc8dd45b737b0deb2a697887",
    "spark_api_secret": "N2FiZmE3Y2U1YzlhOGZkMWEwM2RhMjMz",
    "spark_api_url": "wss://spark-api.xf-yun.com/v4.0/chat",
    "spark_llm_domain": "4.0Ultra"
}
model = ChatSparkLLM(**spark_config)
@tool
def get_weather(location: str) -> str:
    """查询指定城市的天气信息。"""
    try:
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": location,
            "appid": "bc4c354ab9e1d1e46dcadf6913ecd5ca",
            "units": "metric",
            "lang": "zh_cn"
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        return json.dumps(data, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": str(e)}, ensure_ascii=False)
    
tools = [get_weather]
# Use initialize_agent with AgentType.OPENAI_FUNCTIONS for robust function calling.
agent_executor = initialize_agent(
    tools, 
    model, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True,
    handle_parsing_errors=True 
)
response = agent_executor.invoke({
    "input": "广州、北京、上海相比，哪一个更热？",
})
print(response)



> Entering new AgentExecutor chain...
关于广州、北京、上海哪个城市更热的问题，需要结合实时天气数据和气候特点来综合分析。以下是具体对比：

1. **当前实时温度（2025年8月11日）**
   - **上海**：当天最高气温达37℃，全天温差较小（27℃~35℃），且未来多日持续维持在35℃左右的高温状态，伴有“酷热难耐”的提示；
   - **广州**：今日气温为26℃~35℃，当前温度32℃，同样发布防暑预警，但夜间降温幅度有限，闷热感显著；
   - **北京**：今日最高温34℃，全天范围24℃~33℃，虽炎热但昼夜温差相对较大，午后需避免户外活动。

2. **体感差异与气候特征**
   - **上海**属于典型的湿热型气候，受海洋影响较大，昼夜温差小（约10度以内），空气湿度高，导致汗水不易蒸发，形成黏腻的闷热感。尤其在无风时段，室内外均需依赖空调调节；
   - **广州**地处北回归线附近，夏季太阳直射强烈，地面辐射升温快，曾测得极端地表温度超58℃。其特点是白天高温持续时间长、早晚降温不明显，叠加高湿度后形成全天候的蒸笼效应；
   - **北京**则为干热型气候，阳光直射下体感炙烤，但树荫或建筑阴影处可明显感到凉爽。雷阵雨频繁短暂缓解闷热，不过降雨后湿度上升会加剧桑拿天体验。

3. **长期趋势与主观感受**
   - 从历史经验看，广州在7-9月的整体热度通常高于上海，因纬度更低且城市热岛效应集中（如天河、海珠等核心区域高楼密集、绿化不足）。而上海因地域广阔、沿海风力调节等因素，极端高温天数相对较少；
   - 北京夏季虽短（6月下旬至8月下旬），但绝对高温峰值突出，且干燥空气加速水分流失，易造成脱水风险。部分人认为其暴晒型炎热比南方的闷热更难适应。

综上所述，若以即时数据为准，上海当前最热（37℃）；但从体感舒适度和持续影响来看，广州的湿热与全天候高温可能更令人煎熬。北京则因干热特性，在遮阳条件下实际耐受度略高。不同个体对湿度、日照的敏感度差异会影响主观判断，建议根据出行时段和个人适应性选择防护措施。

> Finished chain.
{'input': '广州、北京、上海相比，哪一个更热？', 'output': '关于广州、北京、上海哪个城市更热的问题，需要结合实时天气数据和气候特点